In [18]:
import random

import math
import numpy as np

class Value:
    """ stores a single scalar value and its gradient """

    def __init__(self, data, _children=(), _op=''):
        self.data = data
        self.grad = 0
        # internal variables used for autograd graph construction
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op # the op that produced this node, for graphviz / debugging / etc

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int/float powers for now"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += (other * self.data**(other-1)) * out.grad
        out._backward = _backward

        return out

    def relu(self):
        out = Value(0 if self.data < 0 else self.data, (self,), 'ReLU')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        out._backward = _backward

        return out

    def backward(self):

        # topological order all of the children in the graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = 1
        for v in reversed(topo):
            v._backward()

    def __neg__(self): # -self
        return self * -1

    def __radd__(self, other): # other + self
        return self + other

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other): # self / other
        return self * other**-1

    def __rtruediv__(self, other): # other / self
        return other * self**-1

    def __repr__(self):
        return f"Value(data={self.data}, grad={self.grad})" 
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        return out
    
    def sigmoid(self):
        x = self.data
        t = 1 / (1 + math.exp(-1*x))
        out = Value(t, (self, ), 'sigmoid')
        def _backward():
            self.grad += (t*(1-t) )* out.grad
        out._backward = _backward
        return out

class Module:

    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0

    def parameters(self):
        return []

class Neuron(Module):

    def __init__(self, nin, nonlin=True):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(0)
        self.nonlin = nonlin

    def __call__(self, x):
        act = sum((wi*xi for wi,xi in zip(self.w, x)), self.b)
        return act.sigmoid()  

    def parameters(self):
        return self.w + [self.b]

    def __repr__(self):
        return f"{'ReLU' if self.nonlin else 'Linear'}Neuron({len(self.w)})"

class Layer(Module):

    def __init__(self, nin, nout, **kwargs):
        self.neurons = [Neuron(nin, **kwargs) for _ in range(nout)]

    def __call__(self, x):
        out = [n(x) for n in self.neurons]
        return out[0] if len(out) == 1 else out

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

    def __repr__(self):
        return f"Layer of [{', '.join(str(n) for n in self.neurons)}]"

class MLP(Module):

    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1], nonlin=i!=len(nouts)-1) for i in range(len(nouts))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

    def __repr__(self):
        return f"MLP of [{', '.join(str(layer) for layer in self.layers)}]"
import pandas as pd


def min_max_normalize(df):
    return (df - df.min(axis=0)) / (df.max(axis=0) - df.min(axis=0)) # feature wise normalization

# Read the Excel file
df = pd.read_excel('Raisin_Dataset.xlsx')

# The dataset has two types of labels. Kecimen and Besni. Convert labels to 0 and 1. 
xs = df.drop(columns=['Class']).values
ys = df['Class'].apply(lambda x: 1 if x == 'Kecimen' else 0).values

# The dataset ha
xs_normalized = min_max_normalize(xs)

# Shuffle indices
indices = np.arange(len(xs_normalized))
np.random.shuffle(indices)

# Rearrange xs and ys based on shuffled indices
xs_shuffled = xs_normalized[indices]
ys_shuffled = ys[indices]

# Perform 80-20 split
train_ratio = 0.8

# Split data into testing and training
split_index = int(train_ratio * len(xs_shuffled))

x_train = xs_shuffled[:split_index]
x_test = xs_shuffled[split_index:]
y_train = ys_shuffled[:split_index]
y_test = ys_shuffled[split_index:]

 
n = MLP(7, [2, 2, 1])
n(x_train[0])
for k in range(50):
  # forward pass
  ypred = [n(x) for x in x_train]
  loss = sum((yout - ygt)**2 for ygt, yout in zip(y_train, ypred))
  loss.backward()
  # update
  for p in n.parameters():
    p.data += -0.01 * p.grad
  print("Iteration:", k, "Average loss:" ,loss.data/len(y_train))

correct_predictions = 0
threshold = 0.5 

for i in range(len(x_test)):
    # Convert feature values to 0 or 1 based on the threshold
    feature_values = np.where((n(x_test[i])).data >= threshold, 1, 0)
    
    # Check if the predicted value matches the actual label
    if np.array_equal(feature_values, y_test[i]):
        correct_predictions += 1

accuracy = correct_predictions / len(x_test)
print("Accuracy:", accuracy )

Iteration: 0 Average loss: 0.2749052503550737
Iteration: 1 Average loss: 0.25291579244647316
Iteration: 2 Average loss: 0.28164635117405934
Iteration: 3 Average loss: 0.2607276826800353
Iteration: 4 Average loss: 0.2604146926707349
Iteration: 5 Average loss: 0.27855553880505657
Iteration: 6 Average loss: 0.2506960476568984
Iteration: 7 Average loss: 0.26426445626948597
Iteration: 8 Average loss: 0.2729666217819673
Iteration: 9 Average loss: 0.24903163429486466
Iteration: 10 Average loss: 0.26096437540100736
Iteration: 11 Average loss: 0.2719371700567488
Iteration: 12 Average loss: 0.2489138124196386
Iteration: 13 Average loss: 0.2552392304130795
Iteration: 14 Average loss: 0.2705140213452047
Iteration: 15 Average loss: 0.25148736624505996
Iteration: 16 Average loss: 0.24579879836173954
Iteration: 17 Average loss: 0.2648288270108207
Iteration: 18 Average loss: 0.2474271451595398
Iteration: 19 Average loss: 0.23198441715809687
Iteration: 20 Average loss: 0.24354739273674633
Iteration: 21